In [1]:
import pandas as pd
import numpy as np
import copy

from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras import Input, Model

from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from callbacks import CallbackPlot, CallbackSaveLogs

import tensorflow as tf

from data_processor import clean_data, build_datasets
from data_processor import one_hot2char, process_input

%matplotlib
import matplotlib
import matplotlib.pyplot as plt

import sys
import json
import os
from shutil import copyfile
from datetime import datetime

MAX_LEN_TITLE = 67
MAX_LEN_TEXT = 120

# enable memory growth to be able to work with GPU
GPU = tf.config.experimental.get_visible_devices('GPU')[0]
tf.config.experimental.set_memory_growth(GPU, enable=True)

# set tensorflow to work with float64
tf.keras.backend.set_floatx('float64')

# the new line character (\n) is the 'end of sentence', therefore there is no need to add a '[STOP]' character
vocab = 'c-y5i8"j\'fk,theqm:/.wnlrdg0u1 v\n4b97)o36z2axs(p'
vocab = list(vocab) + ['[START]']

Using matplotlib backend: TkAgg


In [2]:

config = {  # dictionary that contains the training set up. Will be saved as a JSON file
    'DIM_VOCAB': len(vocab),
    'DIM_LSTM_LAYER': 512,
    'ENCODER_DEPTH': 2,
    'DECODER_DEPTH': 2,
    'LEARNING_RATE': 0.001,
    'BATCH_SIZE': 32,
    'EPOCHS': 40,
    'SEED': 1,
    'GRAD_VAL_CLIP': 0.5,
    'GRAD_NORM_CLIP': 1,
    'DECAY_AT_10_EPOCHS': 1,
    'DROPOUT': 0.2,
}
tf.random.set_seed(config['SEED'])
data = pd.read_csv('Complete Database.csv', index_col=0)

data = clean_data(data)

config['STEPS_PER_EPOCH'] = int((data.shape[0] - 3000) / config['BATCH_SIZE'])
config['VALIDATION_SAMPLES'] = int(data.shape[0]) - (config['STEPS_PER_EPOCH'] * config['BATCH_SIZE'])
config['VALIDATION_STEPS'] = int(np.floor(config['VALIDATION_SAMPLES'] / config['BATCH_SIZE']))
'''
config['STEPS_PER_EPOCH'] = 5
config['VALIDATION_SAMPLES'] = 100
config['VALIDATION_STEPS'] = 5
'''
# configures the learning rate to be decayed by the value specified at config['DECAY_AT_10_EPOCHS'] at each 10 epochs, but to that gradally at each epoch
learning_rate = ExponentialDecay(initial_learning_rate=config['LEARNING_RATE'], 
                                decay_steps=config['STEPS_PER_EPOCH'], 
                                decay_rate=np.power(config['DECAY_AT_10_EPOCHS'], 1/10), 
                                staircase=True)


 
training_dataset, validation_dataset = build_datasets(data, seed=config['SEED'], validation_samples=config['VALIDATION_SAMPLES'], batch=config['BATCH_SIZE'], vocab=vocab)

#learning_rate = config['LEARNING_RATE']

if 'GRAD_VAL_CLIP' in config:
    optimizer = Adam(learning_rate=learning_rate, clipvalue=config['GRAD_VAL_CLIP'], clipnorm=config['GRAD_NORM_CLIP'])
    print('Using gradient clipping')
else:
    optimizer = Adam(learning_rate=learning_rate)

config['OPTIMIZER'] = optimizer._name

folder_path = 'Training Logs/Training'  # creates folder to save traning logs
if not os.path.exists(folder_path):
        os.makedirs(folder_path)

with open(folder_path + '/config.json', 'w') as json_file: # saves the training configuration as a JSON file
    json.dump(config, json_file, indent=4)

with open(folder_path + '/vocab.json', 'w')  as json_file: # saves the vocab used as a JSON file
    json.dump(vocab, json_file, indent=4)


validation_steps = int(config['VALIDATION_SAMPLES'] / config['BATCH_SIZE'])

loss_plot_settings = {'variables': {'loss': 'Training loss',
                                        'val_loss': 'Validation loss'},
                          'title': 'Losses',
                          'ylabel': 'Epoch Loss'}

last_5_plot_settings = {'variables': {'loss': 'Training loss',
                                        'val_loss': 'Validation loss'},
                          'title': 'Losses',
                          'ylabel': 'Epoch Loss',
                          'last_epochs': 5}

plot_callback = CallbackPlot(folder_path=folder_path,
                            plots_settings=[loss_plot_settings, last_5_plot_settings],
                            title='Losses', share_x=False)

model_checkpoint_callback = ModelCheckpoint(filepath=folder_path + '/trained_model.h5')

csv_logger = CSVLogger(filename=folder_path + '/Training logs.csv', separator=',', append=False)


Using gradient clipping


In [3]:
###### BUILDS MODEL FROM SCRATCH WITH MULTI LAYER LSTM ###########
tf.keras.backend.clear_session() # destroys the current graph

encoder_inputs = Input(shape=(None, config['DIM_VOCAB']), name='encoder_input')
enc_internal_tensor = encoder_inputs

for i in range(config['ENCODER_DEPTH']):
    encoder_LSTM = LSTM(units=config['DIM_LSTM_LAYER'], return_sequences=True, return_state=True, name='encoder_LSTM_' + str(i), dropout=config['DROPOUT'])
    enc_internal_tensor, enc_memory_state, enc_carry_state = encoder_LSTM(enc_internal_tensor) # only the last states are of interest

decoder_inputs = Input(shape=(None, config['DIM_VOCAB']), name='decoder_input')
dec_internal_tensor = decoder_inputs

for i in range(config['DECODER_DEPTH']):
    decoder_LSTM = LSTM(units=config['DIM_LSTM_LAYER'], return_sequences=True, return_state=True, name='decoder_LSTM_' + str(i), dropout=config['DROPOUT'])  # return_state must be set in order to retrieve the internal states in inference model later
    # every LSTM layer in the decoder model have their states initialized with states from last time step from last LSTM layer in the encoder
    dec_internal_tensor, _, _ = decoder_LSTM(dec_internal_tensor, initial_state=[enc_memory_state, enc_carry_state]) 

decoder_output = dec_internal_tensor

dense = Dense(units=config['DIM_VOCAB'], activation='softmax', name='output')
dense_output = dense(decoder_output)

model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=dense_output)

model.compile(optimizer=optimizer, loss='categorical_crossentropy')

In [4]:

history = model.fit(x=training_dataset,
                    epochs=config['EPOCHS'],
                    steps_per_epoch=config['STEPS_PER_EPOCH'],
                    callbacks=[plot_callback, csv_logger, model_checkpoint_callback],
                    validation_data=validation_dataset,
                    validation_steps=config['VALIDATION_STEPS'])
                   

'''
model.save(folder_path + '/trained_model.h5', save_format='h5')
model.save_weights(folder_path + '/trained_model_weights.h5')
plot_model(model, to_file=folder_path + '/model_layout.png', show_shapes=True, show_layer_names=True, rankdir='LR')
'''

timestamp_end = datetime.now().strftime('%d-%b-%y -- %H:%M:%S')

# renames the training folder with the end-of-training timestamp
root, _ = os.path.split(folder_path)

timestamp_end = timestamp_end.replace(':', '-')
os.rename(folder_path, root + '/' + 'Training Session - ' + timestamp_end)

print("Training Succesfully finished.")

Train for 562 steps, validate for 94 steps
Epoch 1/40
562/562 [==============================] - 1548s 3s/step - loss: 0.2993 - val_loss: 0.2419
Epoch 2/40
562/562 [==============================] - 1528s 3s/step - loss: 0.2233 - val_loss: 0.1879
Epoch 3/40
562/562 [==============================] - 1528s 3s/step - loss: 0.1966 - val_loss: 0.1663
Epoch 4/40
562/562 [==============================] - 1528s 3s/step - loss: 0.1790 - val_loss: 0.1443
Epoch 5/40
562/562 [==============================] - 1528s 3s/step - loss: 0.1635 - val_loss: 0.1415
Epoch 6/40
562/562 [==============================] - 1528s 3s/step - loss: 0.1534 - val_loss: 0.1258
Epoch 7/40
562/562 [==============================] - 1528s 3s/step - loss: 0.1480 - val_loss: 0.1169
Epoch 8/40
562/562 [==============================] - 1529s 3s/step - loss: 0.1418 - val_loss: 0.1201
Epoch 9/40
562/562 [==============================] - 1529s 3s/step - loss: 0.1363 - val_loss: 0.1112
Epoch 10/40
562/562 [==================

In [1]:
# creates a dataframe that contains all the characters and their count in all the texts and titles
import pandas as pd
from data_processor import clean_data

data = pd.read_csv('Complete Database.csv', index_col=0)

data = clean_data(data)

concatenated_text = data.Text.str.cat() + data.Title.str.cat()
vocab = set(list(concatenated_text))

char_count = []
for char in vocab:
    char_count.append([char, concatenated_text.count(char)])

df_count = pd.DataFrame(char_count, columns=['Char', 'Count']) # must be 'Count' with capital 'c' to avoid conflict with function 'count' form pandas
df_count.set_index('Char', inplace=True)
df_count.sort_values(by='Count', ascending=True, inplace=True)

In [5]:

model_folder1 = 'Training Logs/Training Session - 03-May-20 -- 04-00-16'
model_folder2 = 'Training Logs/Training Session - 04-May-20 -- 04-41-34'
with open(model_folder1 + '/config.json') as json_file:
    config1 = json.load(json_file)
with open(model_folder2 + '/config.json') as json_file:
    config2 = json.load(json_file)

config1 == config2

True

In [2]:
####### MULTI LSTM LAYER LOADER ############################

model_folder = 'Training Logs/Training Session - 17-May-20 -- 11-42-37'


tf.keras.backend.clear_session() # destroys the current graph

# loads trained model from H5 file. 't' stands for 'trained'
t_model = tf.keras.models.load_model(model_folder + '/trained_model.h5')

# loads model configuration from JSON file
with open(model_folder + '/config.json') as json_file:
    config = json.load(json_file)

# loads vocab from JSON file
with open(model_folder + '/vocab.json') as json_file:
    vocab = json.load(json_file)

# encoder model. 't' stands for 'trained'
t_enc_input = t_model.get_layer('encoder_input').input
t_enc_internal_tensor = t_enc_input

for i in range(config['ENCODER_DEPTH']):
    t_enc_LSTM = t_model.get_layer('encoder_LSTM_' + str(i))
    [t_enc_internal_tensor, t_enc_memory_state, t_enc_carry_state] = t_enc_LSTM(t_enc_internal_tensor)


t_enc_model = Model(inputs=t_enc_input,
                    outputs=[t_enc_memory_state, t_enc_carry_state])

# decoder inputs
# the model needs to have an initial input, and inputs for every state (both memory and carry states) for each LSTM layer in the decoder,
# because it decodes one time step at a time, always using the previous prediction as input
t_dec_input = Input(shape=(None, config['DIM_VOCAB']), name='decoder_input')
t_dec_internal_tensor = t_dec_input

t_dec_memory_state_inputs = []   # list of all input memory state tensors (1 per LSTM layer)
t_dec_carry_state_inputs = []  # list of all input carry state tensors (1 per LSTM layer)

for i in range(config['DECODER_DEPTH']):
    t_dec_memory_state_inputs.append(Input(shape=(config['DIM_LSTM_LAYER']), name='decoder_memory_state_input_' + str(i)))
    t_dec_carry_state_inputs.append(Input(shape=(config['DIM_LSTM_LAYER']), name='decoder_carry_state_input_' + str(i)))

t_dec_memory_state_outputs = [] # list of all output memory state tensors (1 per LSTM layer)
t_dec_carry_state_outputs = [] # list of all output carry state tensors (1 per LSTM layer)

# decoder model
for i in range(config['DECODER_DEPTH']):
    t_dec_LSTM = t_model.get_layer('decoder_LSTM_' + str(i))
    t_dec_internal_tensor, t_dec_memory_state_output, t_dec_carry_state_output = t_dec_LSTM(t_dec_internal_tensor, initial_state=[t_dec_memory_state_inputs[i], t_dec_carry_state_inputs[i]])
    t_dec_memory_state_outputs.append(t_dec_memory_state_output)
    t_dec_carry_state_outputs.append(t_dec_carry_state_output)

t_dec_output = t_dec_internal_tensor # output of last LSTM layer

t_dense_output = t_model.get_layer('output')
t_dec_prediction = t_dense_output(t_dec_output)
t_dec_model = Model(inputs=[t_dec_input] + t_dec_memory_state_inputs + t_dec_carry_state_inputs,
                    outputs=[t_dec_prediction] + t_dec_memory_state_outputs + t_dec_carry_state_outputs)


    

In [3]:
strings = ['captain rex', 'obi-wan kenobi']
char2int = dict((c, i) for i, c in enumerate(vocab))

vec_input_strings = process_input(strings, vocab)

#end_char = np.zeros((len(vocab)), dtype=np.int8) # creates end of sentence character for the decoder
#end_char[char2int['\n']] = 1

vec_sentences = np.zeros((0, MAX_LEN_TEXT, len(vocab)))
for vec_input_string in vec_input_strings:

    vec_input_string = np.expand_dims(vec_input_string, axis=0)
    [enc_memory_state, enc_carry_state] = t_enc_model.predict([vec_input_string]) # get initial states generated by the encoder

    one_hot_char_output = np.zeros((len(vocab)), dtype=np.int8) # creates initial character for the decoder
    one_hot_char_output[char2int['[START]']] = 1
    one_hot_char_output = np.expand_dims(one_hot_char_output, axis=0)
    one_hot_char_output = np.expand_dims(one_hot_char_output, axis=0)

    memory_states = []  # list of all input memory states (1 per LSTM layer)
    carry_states = []   # list of all input carry states (1 per LSTM layer)

    for i in range(config['DECODER_DEPTH']):    # initializes all memory and carry states with states from last layer of the encoder
        memory_states.append(enc_memory_state)
        carry_states.append(enc_carry_state)

    vec_sentence = np.empty((0, len(vocab)))

    for _ in range(MAX_LEN_TEXT):

        outputs = t_dec_model.predict([one_hot_char_output] + memory_states + carry_states) # output contains the output and each layer's memory and carry states
        one_hot_char_output = outputs.pop(0) # separates the output
        memory_states = outputs[: len(outputs)//2]  # separates memory states from all layers
        carry_states = outputs[len(outputs)//2 :]   # separates carry states from all layers
        vec_sentence = np.append(vec_sentence, one_hot_char_output[0], axis=0)
        
    
    vec_sentences = np.append(vec_sentences, [vec_sentence], axis=0)
    
strings = one_hot2char(vec_sentences, vocab)

In [4]:
strings


["captain iis was a malier male who served as a cirettin aomatel in the galen system.\n''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''",
 "the obinite relter was a trae of eooitaled teviu tave toot toot bone ated by the fane oange s oart o  \n'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''"]

In [6]:
import pandas as pd

training_logs_1 = pd.read_csv('Training Logs/Training/Training logs.csv', index_col=0)

training_logs_2 = pd.read_csv('Training Logs/Training Session - 04-May-20 -- 04-41-34/Training logs.csv', index_col=0)


In [7]:
training_logs_1

,loss,val_loss
epoch,,
0,0.238608,0.153786
1,0.150679,0.120578
2,0.128761,0.113731
3,0.121985,0.101752
4,0.113408,0.096178
5,0.111345,0.098579
6,0.108054,0.091458
7,0.102975,0.088828
8,0.100700,0.091857


In [ ]:
from tensorflow.keras.models import load_model

folder_path = 'Training Logs/Training'

model = load_model(folder_path + 'trained_model.h5')

with open(model_folder1 + '/config.json') as json_file:
    config = json.load(json_file)

tf.random.set_seed(config['SEED'])
data = pd.read_csv('Complete Database.csv', index_col=0)

data = clean_data(data)

training_dataset, validation_dataset = build_datasets(data, seed=config['SEED'], validation_samples=config['VALIDATION_SAMPLES'], batch=config['BATCH_SIZE'], vocab=vocab)

history = model.fit(x=training_dataset,
                    epochs=config['EPOCHS'],
                    steps_per_epoch=config['STEPS_PER_EPOCH'],
                    callbacks=[plot_callback, csv_logger, model_checkpoint_callback],
                    validation_data=validation_dataset,
                    validation_steps=config['VALIDATION_STEPS'])


In [5]:
'''
#Sequence to sequence example in Keras (character-level).

This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.

**Summary of the algorithm**

- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    It uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.

**Data download**

[English to French sentence pairs.
](http://www.manythings.org/anki/fra-eng.zip)

[Lots of neat sentence pairs datasets.
](http://www.manythings.org/anki/)

**References**

- [Sequence to Sequence Learning with Neural Networks
   ](https://arxiv.org/abs/1409.3215)
- [Learning Phrase Representations using
    RNN Encoder-Decoder for Statistical Machine Translation
    ](https://arxiv.org/abs/1406.1078)
'''
from __future__ import print_function

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

ModuleNotFoundError: No module named 'tf'

In [2]:
data = pd.read_csv('Complete Database.csv', index_col=0)

data = clean_data(data)

In [22]:
data.Title.str.len().max()

67

In [5]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
data.shape

(11088, 3)